In [ ]:
import os
import pandas as pd
import networkx as nx
# show current directory
print(os.getcwd())

In [ ]:
def load_graph(file_path):
    graph_data = pd.read_csv(file_path, sep='\t', header=None, comment='%', names=['from', 'to', 'weight'])
    graph_data.dropna(inplace=True)
    return graph_data

def remove_cycles_simplified(graph_data):
    G = nx.DiGraph()
    for index, row in graph_data.iterrows():
        G.add_edge(row['from'], row['to'], weight=row['weight'])
    
    removed_edges = []
    while True:
        try:
            cycle = nx.find_cycle(G)
            if not cycle:
                break
            # Remove the first edge in the cycle (simplified approach)
            edge_to_remove = cycle[0]
            G.remove_edge(*edge_to_remove)
            removed_edges.append(edge_to_remove)
        except nx.NetworkXNoCycle:
            break

    return G, removed_edges

# Path to the file
file_path = 'data/out.dimacs10-celegansneural'

# Load the graph data
graph_data = load_graph(file_path)

In [ ]:
# show number of nodes in the graph
print(len(graph_data['from'].unique()))
# show number of edges in the graph
print(len(graph_data))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_indegree_outdegree_distribution(graph_data):
    # Create a directed graph
    G = nx.DiGraph()
    for index, row in graph_data.iterrows():
        G.add_edge(row['from'], row['to'], weight=row['weight'])

    # Calculate in-degree and out-degree for each node
    indegrees = dict(G.in_degree())
    outdegrees = dict(G.out_degree())

    # Prepare data for plotting
    indegree_values = np.array(list(indegrees.values()))
    outdegree_values = np.array(list(outdegrees.values()))

    # Plotting
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    hist, xedges, yedges = np.histogram2d(indegree_values, outdegree_values, bins=30)

    xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25, indexing="ij")
    xpos = xpos.ravel()
    ypos = ypos.ravel()
    zpos = 0

    dx = dy = 0.5 * np.ones_like(zpos)
    dz = hist.ravel()

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')

    ax.set_xlabel('In-degree')
    ax.set_ylabel('Out-degree')
    ax.set_zlabel('Frequency')

    plt.show()

# Plot the indegree and outdegree distribution
plot_indegree_outdegree_distribution(graph_data)


In [ ]:
# please show the unique weight values in the graph
print(graph_data['weight'].unique())

In [ ]:
graph_data.head()

In [ ]:
# Create a directed networkx graph from the graph data using nx.from_pandas_edgelist
G_pandas = nx.from_pandas_edgelist(graph_data, 'from', 'to', edge_attr='weight', create_using=nx.DiGraph)

In [ ]:
# Let's convert the pandas DataFrame into a NetworkX Directed Graph (DiGraph)

import networkx as nx

def convert_to_digraph(graph_data):
    G = nx.DiGraph()
    for index, row in graph_data.iterrows():
        G.add_edge(row['from'], row['to'], weight=row['weight'])
    return G

# Convert the pandas DataFrame to a NetworkX DiGraph
G = convert_to_digraph(graph_data)

# Display basic information about the graph
print(G)


In [ ]:
import scipy.io
import numpy as np
import networkx as nx

# Load the .mat file
file_path = 'data/ConnOrdered_040903.mat'
mat = scipy.io.loadmat(file_path)

# Load the keys 'A_init_t_ordered', 'Ag_t_ordered',
A_init_t_ordered = np.array(mat['A_init_t_ordered'].todense())
print('A_init_t_ordered shape:', A_init_t_ordered.shape)
Ag_t_ordered = np.array(mat['Ag_t_ordered'].todense())
print('Ag_t_ordered shape:', Ag_t_ordered.shape)

# Create NetworkX DiGraphs from the adjacency matrices
G_A_init_t_ordered = nx.DiGraph(A_init_t_ordered)
G_Ag_t_ordered = nx.DiGraph(Ag_t_ordered)

# Function to print basic graph info
def print_graph_info(G, name):
    print(f"{name} has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print(f"Graph density of {name}: {nx.density(G)}")
    print(f"Graph {name} is directed: {G.is_directed()}")


In [ ]:
# Print information about all the graphs G, G_pandas, G_A_init_t_ordered, and G_Ag_t_ordered and print a new line after each graph info
print_graph_info(G, 'G')
print()
print_graph_info(G_pandas, 'G_pandas')
print()
print_graph_info(G_A_init_t_ordered, 'G_A_init_t_ordered')
print()
print_graph_info(G_Ag_t_ordered, 'G_Ag_t_ordered')
print()

In [ ]:
# Compare with the previously loaded connectome data
def compare_graphs(G1, G2):
    print(f"Graph 1 has {G1.number_of_nodes()} nodes and {G1.number_of_edges()} edges.")
    print(f"Graph 2 has {G2.number_of_nodes()} nodes and {G2.number_of_edges()} edges.")
    common_edges = len(set(G1.edges()).intersection(set(G2.edges())))
    print(f"Number of common edges: {common_edges}")

# Assuming G is the previously loaded connectome data graph
compare_graphs(G, G_pandas)

In [ ]:
import networkx as nx

def are_digraphs_equivalent(G1, G2):
    # Check if both graphs have the same set of nodes
    if set(G1.nodes) != set(G2.nodes):
        return False
    
    # Check if both graphs have the same set of edges
    if set(G1.edges) != set(G2.edges):
        return False
    
    # Check if all corresponding edges have the same weights
    for (u, v) in G1.edges:
        if G1[u][v]['weight'] != G2[u][v]['weight']:
            return False
    
    return True

# Example usage
# Assuming G_A_init_t_ordered and G_Ag_t_ordered are your graphs
G1 = G_A_init_t_ordered  # Replace with your first graph
G2 = G_Ag_t_ordered      # Replace with your second graph

equivalent = are_digraphs_equivalent(G1, G2)
print("The graphs are equivalent:", equivalent)


In [ ]:
equivalent = are_digraphs_equivalent(G, G_pandas)
print("The graphs are equivalent:", equivalent)


In [ ]:
# Check if the graphs are DAGs
def check_if_dag(G, name):
    is_dag = nx.is_directed_acyclic_graph(G)
    print(f"{name} is a DAG: {is_dag}")

# Checking all four graphs
check_if_dag(G, "G")
check_if_dag(G_pandas, "G_pandas")
check_if_dag(G_A_init_t_ordered, "G_A_init_t_ordered (G1)")
check_if_dag(G_Ag_t_ordered, "G_Ag_t_ordered (G2)")

In [ ]:
import igraph as ig
g = ig.Graph(n=10, edges=[[0, 1], [0, 5], [5, 3], [3, 0]])
g.feedback_arc_set()

In [ ]:
# Function to remove cycles using a DFS-based approach
def remove_cycles_dfs(G):
    def dfs(v, visited, stack, removed_edges):
        visited[v] = True
        stack[v] = True
        
        for neighbor in list(G[v]):
            if not visited[neighbor]:
                if dfs(neighbor, visited, stack, removed_edges):
                    return True
            elif stack[neighbor]:
                # Found a back edge
                G.remove_edge(v, neighbor)
                removed_edges.append((v, neighbor))
        
        stack[v] = False
        return False

    visited = {node: False for node in G}
    stack = {node: False for node in G}
    removed_edges = []

    for node in G:
        if not visited[node]:
            dfs(node, visited, stack, removed_edges)
    
    return G, removed_edges

# Create a copy of the graph to preserve the original
G_no_cycles = G.copy()
G_no_cycles, removed_edges = remove_cycles_dfs(G_no_cycles)

# Check if the modified graph has no cycles
has_cycle = nx.is_directed_acyclic_graph(G_no_cycles)

# Print the number of removed edges and whether the graph has cycles
print(f"Number of removed edges: {len(removed_edges)}")
print(f"Graph has cycles: {not has_cycle}")

# Verify if the modified graph is acyclic
is_acyclic = nx.is_directed_acyclic_graph(G_no_cycles)
# Print the result
print(f"The modified graph is acyclic: {is_acyclic}")

In [ ]:
# Function to convert NetworkX DiGraph to igraph Graph
def nx_to_igraph(G):
    # see: https://networkx.org/documentation/stable/auto_examples/external/plot_igraph.html
    ig_graph = ig.Graph.from_networkx(G)
    return ig_graph

# Function to convert igraph Graph to NetworkX DiGraph
def igraph_to_nx(ig_graph):
    G = nx.DiGraph()
    names = ig_graph.vs['_nx_name']
    G.add_nodes_from(names)
    G.add_edges_from([(names[e.source], names[e.target]) for e in ig_graph.es])
    for e in ig_graph.es:
        G[names[e.source]][names[e.target]]['weight'] = e['weight']
    return G

# Function to convert a graph to a DAG by removing the minimum feedback arc set using igraph
def convert_to_dag_igraph(G):
    ig_graph = nx_to_igraph(G)
    # see: https://igraph.org/python/doc/api/igraph._igraph.GraphBase.html#feedback_arc_set
    feedback_arcs = ig_graph.feedback_arc_set(weights='weight')
    # see: https://python.igraph.org/en/stable/tutorial.html#adding-deleting-vertices-and-edges
    ig_graph.delete_edges(feedback_arcs)
    return igraph_to_nx(ig_graph), feedback_arcs

# Convert all four graphs to DAGs
G_dag, removed_edges_G = convert_to_dag_igraph(G.copy())
G_pandas_dag, removed_edges_G_pandas = convert_to_dag_igraph(G_pandas.copy())
G_A_init_t_ordered_dag, removed_edges_G1 = convert_to_dag_igraph(G_A_init_t_ordered.copy())
G_Ag_t_ordered_dag, removed_edges_G2 = convert_to_dag_igraph(G_Ag_t_ordered.copy())

# Print the number of removed edges for each graph
print(f"Number of edges removed from G: {len(removed_edges_G)}")
print(f"Number of edges removed from G_pandas: {len(removed_edges_G_pandas)}")
print(f"Number of edges removed from G_A_init_t_ordered (G1): {len(removed_edges_G1)}")
print(f"Number of edges removed from G_Ag_t_ordered (G2): {len(removed_edges_G2)}")

# Check if the resulting graphs are DAGs
def check_if_dag(G, name):
    is_dag = nx.is_directed_acyclic_graph(G)
    print(f"{name} is a DAG: {is_dag}")

print()
check_if_dag(G_dag, "G_dag")
print()
check_if_dag(G_pandas_dag, "G_pandas_dag")
print()
check_if_dag(G_no_cycles, "G_no_cycles")
print()
check_if_dag(G_A_init_t_ordered_dag, "G_A_init_t_ordered_dag (G1)")
print()
check_if_dag(G_Ag_t_ordered_dag, "G_Ag_t_ordered_dag (G2)")

In [ ]:
# now compare if G_dag and G_no_cycles are equivalent using the previously defined function
equivalent = are_digraphs_equivalent(G_dag, G_no_cycles)
print("The graphs are equivalent:", equivalent)

In [ ]:
print()
print_graph_info(G_dag, "G_dag")
print()
print_graph_info(G_pandas_dag, "G_pandas_dag")
print()
print_graph_info(G_no_cycles, "G_no_cycles")
print()
print_graph_info(G_A_init_t_ordered_dag, "G_A_init_t_ordered_dag (G1)")
print()
print_graph_info(G_Ag_t_ordered_dag, "G_Ag_t_ordered_dag (G2)")